<div align="center">
<b>PhD Dissertation Report</b>
<br><br>
David Noel<br>
Nova Southeastern University<br>
CISD 901: Doctoral Dissertation<br><br>
May, 2023
<br>
<br>
<b>Committee:</b> Dr. Sumitra Mukherjee (Chair), Dr. Michael Laszlo, Dr. Frank Mitropoulos
</div>

# Analysis of Spatial Invariance in Convolutional Neural Networks

1.	Does data augmentation lead to more spatially invariant and robust networks?
2.	Are deep architectures more spatially invariant than wide architectures? Did a particular architecture perform better?
3.	Does generative data augmentation lead to more diverse samples than reinforcement learning techniques? Policy 2 vs. Policy 3.
4.	Do larger datasets increase spatial invariance? (Policy 2,4,5 vs. Policy 1 and 3)
5.	Do stacking data augmentation methods lead to more robust networks? Policy 5.
6.	Does combining synthesized samples with original unaugmented samples improve invariance in CNNs? Policy 4
7.	Do synthesized samples improve invariance? Policy 3
8.	Do Stochastic techniques based on Reinforcement learning techniques improve invariance? Policy 2
9.	Does the number of parameters in a CNN affect its spatial invariance as measured by its test accuracy?


# Imports

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.interpolate import make_interp_spline
from pathlib import Path
import datetime
import os
import glob

# Hyperparameters

In [ ]:
DATASET = "cifar10" # "cifar10", "mnist", "fmnist
NETWORK = "basic"  # "resnet", "inception", "basic"
FOLDER = "." + os.sep + DATASET + os.sep + NETWORK
HISTORY_FILE = FOLDER + os.sep + f'{DATASET}_{NETWORK}_history.csv'
METRICS_FILE = FOLDER + os.sep + f'{DATASET}_{NETWORK}_metrics.csv'
NUMBER_POLICIES = 5
NETWORK_TITLE = "ResNet-50" if NETWORK == "resnet" else "InceptionV3" if NETWORK == "inception" else "the Base Model"
DATASET_TITLE = DATASET.upper()
NUM_EXPERIMENTS = 45

In [ ]:
assert DATASET in ["cifar10", "mnist", "fmnist"], "DATASET must be cifar10, mnist or fmnist"
assert NETWORK in ["resnet", "inception", "basic"], "NETWORK must be resnet, inception, or basic"

# Load history and metrics

#### History summary

In [ ]:
history = pd.read_csv(HISTORY_FILE)
print("Total records:", len(history))
history.head()

#### Metrics summary

In [ ]:
# Combine all metrics files into pandas dataframe
all_metric_files = Path("./").rglob('*metrics.csv')
metrics = pd.concat((pd.read_csv(f) for f in all_metric_files), ignore_index=True)
assert len(metrics) == NUM_EXPERIMENTS, "Number of experiments must be 45"
metrics.head()

#### Dataset and Network Summary

In [ ]:
df = metrics.where((metrics['dataset'] == DATASET) & (metrics['model']==NETWORK)).dropna()
df['policy'] = df['policy'].astype(int)
df

#### Percent improvement

In [ ]:
# compute percentate change in accuracy relative to baseline
df['delta(%)'] = df.groupby('model')['accuracy'].apply(lambda x: x.div(x.iloc[0]).sub(1).mul(100)).round(2)

# set decimal places
df.precision = df.precision.round(4)
df.recall = df.recall.round(4)
df.f1score = df.f1score.round(4)
df.latency = df.latency.round(2)
df = df.reset_index(drop=True)

# save to csv
df.to_csv(FOLDER + "/{}_{}_pct_change.csv".format(DATASET, NETWORK), index=False)
df

#### Epoch summary

In [ ]:
# show number of epochs for each policy
epochs = history.groupby('policy').count().rename(columns={'loss': 'epochs'})
epochs['dataset'] = DATASET
epochs['model'] = NETWORK
epochs = epochs.reset_index()[['dataset','model','policy', 'epochs']]
epochs.to_csv(FOLDER + "/{}_{}_epochs.csv".format(DATASET, NETWORK), index=False)
epochs

#### Extract and plot policy histories

In [ ]:
# get max loss in history. will be used to set y-axis limit in loss plot
max_loss = max(history['val_loss'])

In [ ]:
# Extract the history for each policy
policies = []
for i in range(1, NUMBER_POLICIES + 1):
    policies.append(history[history['policy'] == i])

In [ ]:
def plot_history(history, policy_num):
    epochs = range(1, len(history.loss) + 1)
    # plot loss during training
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(f"Policy {policy_num} for {DATASET_TITLE} on {NETWORK_TITLE}")
    ax1.set_title("Model Loss", fontsize=10)
    ax1.plot(epochs, history.loss, "--")
    ax1.plot(epochs, history.val_loss, "--")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.set_ylim(top=max_loss + 0.1)
    ax1.legend(["training", "validation"], loc="best")
    # plot accuracy during training
    ax2.set_title("Model Accuracy", fontsize=10)
    ax2.plot(epochs, history.accuracy, "--")
    ax2.plot(epochs, history.val_accuracy, "--")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    ax2.legend(["training", "validation"], loc="best")
    plt.tight_layout()
    plt.savefig(FOLDER + f"/{DATASET}_{NETWORK}_policy{policy_num}_plot.png")
    plt.show()
    plt.close()

In [ ]:
# Plot the history for each policy
for i, p in enumerate(policies):
    plot_history(p, i+1)

In [ ]:
def plot_all_policies(policies):
    #epochs = range(1, len(policies[0].loss) + 1)
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10, 6))
    fig.suptitle(f"All policies for {DATASET_TITLE} on {NETWORK_TITLE}")
    ax1.set_title("Training Loss", fontsize=10)
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax2.set_title("Validation Accuracy", fontsize=10)
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    for i, p in enumerate(policies):
        epochs = range(1, len(p.loss) + 1)
        ax1.plot(epochs, p.loss, "--", label=f"policy {i+1}")
        ax2.plot(epochs, p.val_accuracy, label=f"policy {i+1}")
    ax1.legend(loc="best")
    ax2.legend(loc="best")
    plt.tight_layout()
    plt.savefig(FOLDER + "/{}_{}_all_policies_plot.png".format(DATASET, NETWORK))
    plt.show()
    plt.close()

In [ ]:
# plot history for all policies
plot_all_policies(policies)

### Latency
#### Training latency per policy

In [ ]:
# plot latency for DATASET and NETWORK
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(df.policy, df.latency)
ax.bar_label(bars, fmt="%.0f", padding=1, label_type="center")
ax.set_title(f"Training latency for {DATASET_TITLE} on {NETWORK_TITLE}")
ax.set_xlabel("Latency (seconds)")
ax.set_ylabel("Policy")
plt.tight_layout()
plt.show()

#### Total training latency

In [ ]:
# get time delta from seconds
def get_time(num_seconds):
    h, m, s = str(datetime.timedelta(seconds=int(num_seconds))).split(':')
    return f'{h} hours, {m} minutes and {s} seconds'

In [ ]:
# compute total training latency in hours, minutes and seconds
train_time = metrics.pivot_table(index='model', columns='dataset', values='latency', aggfunc='sum')

print("BASIC MODEL TRAINING TIME")
seconds = 0
for i in train_time.columns:
    seconds += train_time[i].basic
    time_delta = get_time(train_time[i].basic)
    print(f'{i}: {time_delta}')
print("Total train time: ", get_time(seconds))

print("\nINCEPTION TRAINING TIME")
seconds = 0
for i in train_time.columns:
    seconds += train_time[i].inception
    time_delta = get_time(train_time[i].inception)
    print(f'{i}: {time_delta}')
print("Total train time: ", get_time(seconds))

print("\nRESNET TRAINING TIME")
seconds = 0
for i in train_time.columns:
    seconds += train_time[i].resnet
    time_delta = get_time(train_time[i].resnet)
    print(f'{i}: {time_delta}')
print("Total train time: ", get_time(seconds))

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
sns.barplot(x = 'model',
            y = 'latency',
            hue = 'dataset',
            data = metrics,
            palette = "Blues",
            errwidth=0,
            estimator=sum,
            edgecolor = "w")
ax.set_title("Training Times Across Different Models")
ax.set(xlabel='Model', ylabel='Latency (seconds)')
ax.legend(title="Dataset",loc='best')
for i in ax.containers:
    ax.bar_label(i,fmt="%.0f", padding=1,label_type='center')
plt.show()

#### Accuracy and F-1 Score

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
bar1 = ax.bar(df.policy - 0.2, df.accuracy, 0.4, label = 'Accuracy')
bar2 = ax.bar(df.policy + 0.2, df.f1score, 0.4, label = 'F-1 score',color=sns.color_palette("Blues")[1])
ax.bar_label(bar1, fmt="%.4f", padding=1, label_type='center')
ax.bar_label(bar2, fmt="%.4f", padding=1, label_type='center')
ax.set_xlabel("Policy")
ax.set_ylabel("Accuracy/F-1 score")
ax.set_title(f"Accuracy and F-1 score for {DATASET_TITLE} on {NETWORK_TITLE}")
ax.legend(loc='best')
plt.tight_layout()
plt.show()

#### Dataset Accuracy

In [ ]:
#group metrics by highest accuracy
max_acc = metrics.groupby(['dataset','model'])['accuracy'].max().reset_index()
max_acc = max_acc.merge(metrics, on=['dataset','model','accuracy'])
max_acc

In [ ]:
# show chart of highest accuracy for each dataset and model
fig, ax = plt.subplots(figsize = (10,6))
sns.barplot(x = 'model',
            y = 'accuracy',
            hue = 'dataset',
            data = max_acc,
            palette = "Blues",
            errwidth=0,
            edgecolor = "w")
ax.set_title("Dataset Accuracy on Each Model")
for i in ax.containers:
    ax.bar_label(i,fmt="%.4f", padding=1,label_type='center')
plt.show()

#### Policy Accuracy on Dataset

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
sns.barplot(x = 'dataset',
            y = 'accuracy',
            hue = 'policy',
            data = metrics,
            palette = "Blues",
            errwidth=0,
            edgecolor = "w")
ax.set_title("Policy Accuracy on Each Dataset")
for i in ax.containers:
    ax.bar_label(i,fmt="%.4f", padding=1,label_type='center',rotation=90)
plt.show()

#### Policy Accuracy on Model

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
sns.barplot(x = 'model',
            y = 'accuracy',
            hue = 'policy',
            data = metrics,
            palette = "Blues",
            errwidth=0,
            edgecolor = "w")
ax.set_title("Policy Accuracy on Each Model")
for i in ax.containers:
    ax.bar_label(i,fmt="%.4f", padding=1, label_type='center',rotation=90)
plt.show()

#### Latency per Policy

In [ ]:
# Plot the average latency for each policy
fig, ax = plt.subplots(figsize = (10,6))
sns.barplot(x = 'policy',
            y = 'latency',
            data = metrics,
            palette = "Blues",
            errwidth=0,
            edgecolor = "w")
ax.set_title("Average Latency for Each Policy")
ax.set(xlabel='policy', ylabel='latency (seconds)')
for i in ax.containers:
    ax.bar_label(i,fmt="%.0f", padding=1,label_type='center')
plt.show()

#### Accuracy based on number of parameters

In [ ]:
param_stats = metrics.groupby(['model','params'])['accuracy'].describe()
param_stats = param_stats.drop(['count'],axis=1)
param_stats['mean'] = param_stats['mean'].round(4)
param_stats['std'] = param_stats['std'].round(4)
param_stats['min'] = param_stats['min'].round(4)
param_stats['max'] = param_stats['max'].round(4)
param_stats = param_stats.rename(columns={'50%': 'median'}).round(4)
param_stats = param_stats.reset_index()
param_stats

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
ax = sns.boxplot(x=metrics.model, y=metrics.accuracy, hue=metrics.params, data=metrics, palette="Blues")
ax.set_title("Accuracy per Total Parameters")
ax.set_xlabel("model")
ax.set_ylabel("accuracy")
ax.legend(title="Total Parameters",loc='best')

lines = ax.get_lines()
categories = ax.get_xticks()

for cat in categories:
    y = lines[4+cat*6].get_ydata()[0]
    ax.text(
        cat, 
        y, 
        f'{y}', 
        ha='center', 
        va='center', 
        color='white',
        bbox=dict(facecolor='#828282', edgecolor='#828282')
    )

plt.show()